In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.8 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [5]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [7]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-05-28 05:35:09,481] A new study created in memory with name: no-name-7301c538-6187-4f67-a1b0-fa25ba05bd55
[I 2025-05-28 05:35:10,936] Trial 0 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 119, 'max_depth': 12}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-05-28 05:35:11,512] Trial 1 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 50, 'max_depth': 10}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-05-28 05:35:12,567] Trial 2 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 85, 'max_depth': 10}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-05-28 05:35:13,826] Trial 3 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 116, 'max_depth': 12}. Best is trial 3 with value: 0.7765363128491621.
[I 2025-05-28 05:35:16,014] Trial 4 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 165, 'max_depth': 9}. Best is trial 3 with value: 0.7765363

In [9]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7783985102420856
Best hyperparameters: {'n_estimators': 135, 'max_depth': 7}


In [10]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.75


In [11]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [12]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-05-28 05:45:03,660] A new study created in memory with name: no-name-8b0332fb-8b9f-49f1-8cbe-ef2640475c93
[I 2025-05-28 05:45:04,779] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-05-28 05:45:06,370] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-05-28 05:45:06,860] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-05-28 05:45:07,419] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-05-28 05:45:07,958] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [13]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [14]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


#Optuna Visualizations

In [15]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [16]:
# 1. Optimization History
plot_optimization_history(study).show()

In [17]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [18]:
# 3. Slice Plot
plot_slice(study).show()

In [19]:
# 4. Contour Plot
plot_contour(study).show()

In [20]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

#Optimizing Multiple ML Models

In [21]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [22]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [23]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-05-28 06:16:29,759] A new study created in memory with name: no-name-ee71f327-7c0f-4400-99c3-758eb5f4af3c
[I 2025-05-28 06:16:34,262] Trial 0 finished with value: 0.7616387337057727 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 236, 'learning_rate': 0.04254437537009751, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-05-28 06:16:34,289] Trial 1 finished with value: 0.7411545623836128 and parameters: {'classifier': 'SVM', 'C': 0.6705499459367588, 'kernel': 'poly', 'gamma': 'auto'}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-05-28 06:16:35,473] Trial 2 finished with value: 0.7430167597765364 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 83, 'learning_rate': 0.013509202291156127, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-05-28 06:16:35,503] Trial 3 finished with value: 0.78398

In [24]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'RandomForest', 'n_estimators': 58, 'max_depth': 17, 'min_samples_split': 8, 'min_samples_leaf': 9, 'bootstrap': False}
Best trial accuracy: 0.7858472998137803


In [25]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.761639,2025-05-28 06:16:29.763978,2025-05-28 06:16:34.261789,0 days 00:00:04.497811,NaN,NaN,GradientBoosting,NaN,NaN,0.042544,6.0,8.0,10.0,236.0,COMPLETE
1,1,0.741155,2025-05-28 06:16:34.263015,2025-05-28 06:16:34.289216,0 days 00:00:00.026201,0.670550,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.743017,2025-05-28 06:16:34.290331,2025-05-28 06:16:35.473338,0 days 00:00:01.183007,NaN,NaN,GradientBoosting,NaN,NaN,0.013509,10.0,2.0,10.0,83.0,COMPLETE
3,3,0.783985,2025-05-28 06:16:35.474476,2025-05-28 06:16:35.503522,0 days 00:00:00.029046,1.496307,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.731844,2025-05-28 06:16:35.504425,2025-05-28 06:16:37.730383,0 days 00:00:02.225958,NaN,NaN,GradientBoosting,NaN,NaN,0.060466,17.0,2.0,6.0,98.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.737430,2025-05-28 06:17:15.701180,2025-05-28 06:17:15.744097,0 days 00:00:00.042917,21.025477,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.782123,2025-05-28 06:17:15.745108,2025-05-28 06:17:16.008013,0 days 00:00:00.262905,NaN,False,RandomForest,NaN,NaN,NaN,19.0,9.0,4.0,56.0,COMPLETE
97,97,0.785847,2025-05-28 06:17:16.009327,2025-05-28 06:17:16.238232,0 days 00:00:00.228905,48.567314,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-05-28 06:17:16.239358,2025-05-28 06:17:16.566799,0 days 00:00:00.327441,72.167044,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [27]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,62
RandomForest,27
GradientBoosting,11


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.748603
RandomForest,0.770398
SVM,0.766535


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()